<a href="https://colab.research.google.com/github/elizabethavargas/Deepfake-Detection/blob/main/deep_learning_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detecting Deepfakes with DINOv2

## Load & Transform Dataset


In [2]:
!pip install datasets

import os, zipfile, subprocess, glob, shutil
from huggingface_hub import snapshot_download
from datasets import load_dataset

In [3]:
# load dataset from huggingface, due to unusual file storage, it needs to be loaded manually
repo = snapshot_download("pujanpaudel/deepfake_face_classification", repo_type="dataset")
out = os.path.join(repo, "extracted")
os.makedirs(out, exist_ok=True)

def extract(src, dst):
    os.makedirs(dst, exist_ok=True)
    if src.endswith(".zip"):
        with zipfile.ZipFile(src) as z: z.extractall(dst)
    elif src.endswith(".rar"):
        if not shutil.which("unrar"):
            subprocess.run(["apt-get","update"]); subprocess.run(["apt-get","install","-y","unrar-free"])
        subprocess.run(["unrar","x","-o+",src,dst])

def find_root(p):
    imgs = glob.glob(f"{p}/**/*.[jJpP]*[gG]", recursive=True)
    class_dir = os.path.dirname(imgs[0])
    return os.path.dirname(class_dir)

splits = {"train":"train.rar","validation":"val.zip","test":"test.zip"}
datasets = {}

for split, fname in splits.items():
    path = os.path.join(repo, fname)
    if not os.path.exists(path): continue
    dst = os.path.join(out, split)
    extract(path, dst)
    root = find_root(dst)
    datasets[split] = load_dataset("imagefolder", data_dir=root)["train"]
    print(split, datasets[split])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/25696 [00:00<?, ?it/s]

train Dataset({
    features: ['image', 'label'],
    num_rows: 25696
})


Resolving data files:   0%|          | 0/3212 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

validation Dataset({
    features: ['image', 'label'],
    num_rows: 3212
})


Resolving data files:   0%|          | 0/3212 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

test Dataset({
    features: ['image', 'label'],
    num_rows: 3212
})


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
from torchvision import transforms

preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

def preprocess_dataset(example):
    example["pixel_values"] = preprocess(example["image"])
    return example

train_dataset = datasets['train'].map(preprocess_dataset)
val_dataset   = datasets['validation'].map(preprocess_dataset)
#test_dataset  = datasets['test'].map(preprocess_dataset)


Map:   0%|          | 0/3212 [00:00<?, ? examples/s]

## Finetune DINO-v2-small


In [12]:
from transformers import AutoModelForImageClassification
import torch

# Load the pre-trained DINOv2 model
model = AutoModelForImageClassification.from_pretrained("facebook/dinov2-small", num_labels=2)

Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at facebook/dinov2-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_steps=50,
    dataloader_num_workers=12,
    fp16=True,
    optim="adamw_torch",


)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()


Epoch,Training Loss,Validation Loss


In [11]:
from sklearn.metrics import accuracy_score

predictions = trainer.predict(val_dataset)

accuracy = accuracy_score(predictions.label_ids, predictions.predictions.argmax(-1))
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9763


### Save Model

In [10]:
from google.colab import drive
drive.mount('/content/drive')

import torch
torch.save(model.state_dict(), '/content/drive/MyDrive/deepfake_dino_model_full_64.pth')
print("Model saved to Google Drive.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model saved to Google Drive.


### Load Saved Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Define the path where the model is saved in Google Drive
model_save_path = '/content/drive/MyDrive/deepfake_dino_model.pth'

# Load the state dictionary
model.load_state_dict(torch.load(model_save_path, map_location=torch.device('cpu')))

# Move the model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Mounted at /content/drive


Dinov2ForImageClassification(
  (dinov2): Dinov2Model(
    (embeddings): Dinov2Embeddings(
      (patch_embeddings): Dinov2PatchEmbeddings(
        (projection): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Dinov2Encoder(
      (layer): ModuleList(
        (0-11): 12 x Dinov2Layer(
          (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
          (attention): Dinov2Attention(
            (attention): Dinov2SelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
            )
            (output): Dinov2SelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (layer_scale1): Dinov2Laye

## Visualize

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import cv2
import random
from transformers import AutoModelForImageClassification, AutoConfig

In [ ]:
# create new config with eager attention & reload model
config = AutoConfig.from_pretrained("facebook/dinov2-small")
config.attn_implementation = "eager"
config.output_attentions = True
config.num_labels = 2

from google.colab import drive
drive.mount('/content/drive')

model = AutoModelForImageClassification.from_pretrained(
    "facebook/dinov2-small",
    config=config
)

model_save_path = '/content/drive/MyDrive/deepfake_dino_model.pth'
model.load_state_dict(torch.load(model_save_path))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

# set up hooks
for block in model.dinov2.encoder.layer:
    attn = block.attention.attention
    if hasattr(attn, "fused_attn"):
        attn.fused_attn = False
    if hasattr(attn, "use_fused_attn"):
        attn.use_fused_attn = False


attention_weights = []
def get_attn_hook(module, input, output):
    # output = (context_layer, attention_probs)
    attention_weights.append(output[1])

# Remove old hooks
for block in model.dinov2.encoder.layer:
    if hasattr(block.attention.attention, '_forward_hooks'):
        block.attention.attention._forward_hooks.clear()

# Attach hook to last block
hook = model.dinov2.encoder.layer[-1].attention.attention.register_forward_hook(
    get_attn_hook
)

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

Mounted at /content/drive


model.safetensors:   0%|          | 0.00/88.2M [00:00<?, ?B/s]

Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at facebook/dinov2-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def visualize_random_samples(model, dataset, device, num_images=3):
    model.eval()

    # 1. Pick Random Indices specifically for FAKES
    print("🎲 Picking random images...")

    # The dataset from load_dataset('imagefolder') does not have a .targets attribute directly.
    # We need to extract labels first or use the 'label' feature.
    # Assuming 'label' is present in the dataset items.
    fake_indices = [i for i, item in enumerate(dataset) if item['label'] == 1]

    if len(fake_indices) < num_images:
        print(f"⚠️ Not enough fake images found ({len(fake_indices)}). Visualizing all available fake images.")
        if len(fake_indices) == 0:
            print("No fake images available to visualize.")
            return
        selected_indices = fake_indices
    else:
        # Randomly sample from the fake list
        selected_indices = random.sample(fake_indices, num_images)

    print(f"✨ Analyzing Indices: {selected_indices}")

    for idx in selected_indices:
        # Get single item
        item = dataset[idx]
        label = item['label']

        image = None
        # Try to get the preprocessed tensor first
        if 'pixel_values' in item:
            if isinstance(item['pixel_values'], torch.Tensor):
                image = item['pixel_values']
            elif isinstance(item['pixel_values'], list) and len(item['pixel_values']) == 1 and isinstance(item['pixel_values'][0], torch.Tensor):
                image = item['pixel_values'][0]

        # If image is still not a tensor, try to re-preprocess the original image
        if not isinstance(image, torch.Tensor):
            if 'image' in item:
                try:
                    global preprocess # Ensure preprocess is accessible from global scope
                    image = preprocess(item['image'])
                except Exception as e:
                    print(f"❌ Failed to re-preprocess original image for index {idx}: {e}. Skipping visualization.")
                    continue
            else:
                print(f"❌ Image data for index {idx} is not a torch.Tensor and original image is unavailable. Skipping visualization.")
                continue

        print('images done')

        # Prepare batch of size 1
        images = image.unsqueeze(0).to(device) # [1, 3, 224, 224] for DINOv2 small

        attention_weights.clear()

        with torch.no_grad():
            outputs = model(images)
            pred_label = outputs.logits.argmax(1).item() # Access logits from model output

        if not attention_weights:
            print(f"❌ Failed to capture attention weights for index {idx}. Skipping visualization.")
            continue

        # Get Matrix
        # attn_mat shape: (batch_size, num_heads, num_tokens, num_tokens)
        attn_mat = attention_weights[0].cpu() # [1, Heads, Tokens, Tokens]

        # --- PROCESS ATTENTION ---
        patch_size = model.dinov2.config.patch_size # Dynamically get patch size from model config
        image_size = model.dinov2.config.image_size # Dynamically get image size from model config
        w_featmap = images.shape[2] // patch_size
        h_featmap = images.shape[3] // patch_size
        # CLS token is at index 0. Registers are not present in dinov2-small by default.
        # So we take attention from CLS token to patch tokens (index 1 onwards).
        cls_token_idx = 0
        patch_tokens_start_idx = 1

        # Extract & Mean
        # attn_mat shape: (batch_size, num_heads, num_tokens, num_tokens)
        # We want the attention from the CLS token (index 0) to all patch tokens (index 1 onwards)
        cls_attn = attn_mat[0, :, cls_token_idx, patch_tokens_start_idx:] # [num_heads, num_patch_tokens]
        cls_attn = cls_attn.mean(dim=0) # [num_patch_tokens]

        # Reshape
        if cls_attn.shape[0] == w_featmap * h_featmap:
            cls_attn = cls_attn.reshape(h_featmap, w_featmap)
        else:
            # Fallback for unexpected token count, should ideally match w_featmap * h_featmap
            print(f"Warning: Attention map token count ({cls_attn.shape[0]}) does not match expected featmap size ({w_featmap * h_featmap}). Attempting square reshape.")
            side = int(cls_attn.shape[0]**0.5)
            if side * side == cls_attn.shape[0]:
                cls_attn = cls_attn.reshape(side, side)
                cls_attn = torch.tensor(cv2.resize(cls_attn.numpy(), (w_featmap, h_featmap), interpolation=cv2.INTER_LINEAR))
            else:
                print("Error: Cannot reshape attention map to square. Skipping visualization for this image.")
                continue

        # Upscale
        cls_attn = F.interpolate(
            cls_attn.unsqueeze(0).unsqueeze(0),
            size=(images.shape[2], images.shape[3]),
            mode='bicubic',
            align_corners=False
        ).squeeze().numpy()

        # --- PLOT ---
        # Unnormalize the image for display
        img_original = images[0].permute(1, 2, 0).cpu().numpy()
        mean = np.array([0.5, 0.5, 0.5]) # Used 0.5 for normalization in preprocess
        std = np.array([0.5, 0.5, 0.5])
        img_original = (img_original * std + mean).clip(0, 1)

        fig, ax = plt.subplots(1, 2, figsize=(10, 5))

        # Title Color: Green if Model caught it, Red if it missed
        status_color = 'green' if pred_label == 1 else 'red'
        status_text = "DETECTED" if pred_label == 1 else "MISSED"

        ax[0].imshow(img_original)
        ax[0].set_title(f"Ground Truth: {'FAKE' if label == 1 else 'REAL'}\nResult: {status_text}", color=status_color, weight='bold')
        ax[0].axis('off')

        ax[1].imshow(img_original)
        ax[1].imshow(cls_attn, cmap='jet', alpha=0.5)
        ax[1].set_title("DINOv2 Attention Map")
        ax[1].axis('off')

        plt.tight_layout()
        plt.show()

# Pass the DATASET (val_dataset) and device
visualize_random_samples(model, val_dataset, device, num_images=3)

🎲 Picking random images...


KeyboardInterrupt: 

In [ ]:
def visualize_sample(model, dataset, device, idx):
    model.eval()

    # Get single item
    item = dataset[idx]
    label = item['label']

    image = None
    # Try to get the preprocessed tensor first
    if 'pixel_values' in item:
      print('pixel')
      if isinstance(item['pixel_values'], torch.Tensor):
          image = item['pixel_values']
          print('A')
      elif isinstance(item['pixel_values'], list) and len(item['pixel_values']) == 1 and isinstance(item['pixel_values'][0], torch.Tensor):
          image = item['pixel_values'][0]
          print('B')

    # If image is still not a tensor, try to re-preprocess the original image
    if not isinstance(image, torch.Tensor):
      print('not tensor')
      if 'image' in item:
          try:
              global preprocess # Ensure preprocess is accessible from global scope
              image = preprocess(item['image'])
          except Exception as e:
              print(f"❌ Failed to re-preprocess original image for index {idx}: {e}. Skipping visualization.")

      else:
          print(f"❌ Image data for index {idx} is not a torch.Tensor and original image is unavailable. Skipping visualization.")


    print('images done')

    # Prepare batch of size 1
    images = image.unsqueeze(0).to(device) # [1, 3, 224, 224] for DINOv2 small

    attention_weights.clear()

    with torch.no_grad():
        outputs = model(images)
        pred_label = outputs.logits.argmax(1).item() # Access logits from model output

    if not attention_weights:
        print(f"❌ Failed to capture attention weights for index {idx}. Skipping visualization.")


    # Get Matrix
    # attn_mat shape: (batch_size, num_heads, num_tokens, num_tokens)
    attn_mat = attention_weights[0].cpu() # [1, Heads, Tokens, Tokens]

    # --- PROCESS ATTENTION ---
    patch_size = model.dinov2.config.patch_size # Dynamically get patch size from model config
    image_size = model.dinov2.config.image_size # Dynamically get image size from model config
    w_featmap = images.shape[2] // patch_size
    h_featmap = images.shape[3] // patch_size
    # CLS token is at index 0. Registers are not present in dinov2-small by default.
    # So we take attention from CLS token to patch tokens (index 1 onwards).
    cls_token_idx = 0
    patch_tokens_start_idx = 1

    # Extract & Mean
    # attn_mat shape: (batch_size, num_heads, num_tokens, num_tokens)
    # We want the attention from the CLS token (index 0) to all patch tokens (index 1 onwards)
    cls_attn = attn_mat[0, :, cls_token_idx, patch_tokens_start_idx:] # [num_heads, num_patch_tokens]
    cls_attn = cls_attn.mean(dim=0) # [num_patch_tokens]

    # Reshape
    if cls_attn.shape[0] == w_featmap * h_featmap:
        cls_attn = cls_attn.reshape(h_featmap, w_featmap)
    else:
        # Fallback for unexpected token count, should ideally match w_featmap * h_featmap
        print(f"Warning: Attention map token count ({cls_attn.shape[0]}) does not match expected featmap size ({w_featmap * h_featmap}). Attempting square reshape.")
        side = int(cls_attn.shape[0]**0.5)
        if side * side == cls_attn.shape[0]:
            cls_attn = cls_attn.reshape(side, side)
            cls_attn = torch.tensor(cv2.resize(cls_attn.numpy(), (w_featmap, h_featmap), interpolation=cv2.INTER_LINEAR))
        else:
            print("Error: Cannot reshape attention map to square. Skipping visualization for this image.")


    # Upscale
    cls_attn = F.interpolate(
        cls_attn.unsqueeze(0).unsqueeze(0),
        size=(images.shape[2], images.shape[3]),
        mode='bicubic',
        align_corners=False
    ).squeeze().numpy()

    # --- PLOT ---
    # Unnormalize the image for display
    img_original = images[0].permute(1, 2, 0).cpu().numpy()
    mean = np.array([0.5, 0.5, 0.5]) # Used 0.5 for normalization in preprocess
    std = np.array([0.5, 0.5, 0.5])
    img_original = (img_original * std + mean).clip(0, 1)

    fig, ax = plt.subplots(1, 2, figsize=(10, 5))

    # Title Color: Green if Model caught it, Red if it missed
    status_color = 'green' if pred_label == 1 else 'red'
    status_text = "DETECTED" if pred_label == 1 else "MISSED"

    ax[0].imshow(img_original)
    ax[0].set_title(f"Ground Truth: {'FAKE' if label == 1 else 'REAL'}\nResult: {status_text}", color=status_color, weight='bold')
    ax[0].axis('off')

    ax[1].imshow(img_original)
    ax[1].imshow(cls_attn, cmap='jet', alpha=0.5)
    ax[1].set_title("DINOv2 Attention Map")
    ax[1].axis('off')

    plt.tight_layout()
    plt.show()

# Pass the DATASET (val_dataset) and device
visualize_sample(model, val_dataset, device, 3)

NameError: name 'model' is not defined

## Ablation: Train head only (vs whole model above)

In [ ]:
from transformers import AutoModelForImageClassification
import torch

# Load the pre-trained DINOv2 model
model = AutoModelForImageClassification.from_pretrained("facebook/dinov2-small", num_labels=2)

In [ ]:
# only fine-tune the last layer
for name, param in model.named_parameters():
    param.requires_grad = False

for name, param in model.named_parameters():
    print(name, param.requires_grad)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_steps=50,

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

### Save Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import torch
torch.save(model.state_dict(), '/content/drive/MyDrive/deepfake_dino_model_head.pth')
print("Model saved to Google Drive.")
